In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


 #READ DATA
 

In [4]:
dataframe = pd.read_csv('../Data/cleaned_data1.csv')


In [13]:
dataframe = pd.read_csv("../DATA/cleaned_data1.csv", names=["tweet", "class"]).iloc[2:,:]
dataframe["class"] = np.where(dataframe["class"] == "Positive",1,0)
dataframe = dataframe.sample(frac = 1)
x= dataframe['tweet'].apply(lambda x : str(x).split()).to_list()
y = dataframe['class'].to_list()


In [14]:
print(dataframe)

                                                   tweet  class
19824  الكتاب مقالات نشرت لاحمد حلمي يستحق ان يشتري ل...      0
22551  جاوبني حفيظ عرض ب مليون اورو زايد اللاعب مانقا...      0
16435  اعلاناتهم المدفوعه بملايين الدولارات صحف اميرك...      0
19788   اتحدي اي واحد يشتري مره اولي يرجع ههه ممتازه مره      0
1136                                              التقشف      0
...                                                  ...    ...
39133  افضل منتجع شرم ذهبت الي المنتجع و كان نظيفا جد...      1
24726  روايه ضعيفه اخر كام صفحه اللي قصه حقيقيه اصلا ...      0
38433  منتجع رايع جدا المنتجع رايع رايع رايع قضينا اي...      1
47365  ابطال اسيا خسر الاتحاد سونغنام جده ورجع بالكاس...      1
18816  الكتاب اقل توقعاتي لموهبه احمد مراد فهو بحق مب...      0

[49864 rows x 2 columns]


In [15]:
print(x[1])

['جاوبني', 'حفيظ', 'عرض', 'ب', 'مليون', 'اورو', 'زايد', 'اللاعب', 'مانقالا', 'حسب', 'الصحف', 'فيشاي', 'يريد', 'اكثر', 'مليون', 'اورو', 'سيبيعه', 'بهذالسعر', 'رايك', 'العلم', 'اللاعب', 'يبلغ', 'العمر', 'سنه']


In [16]:
x_train, x_test, y_train,y_test= train_test_split(x,y,test_size=0.25,random_state=42 , stratify=y)  #same random state 


In [17]:
vec_size = 300    # Word vector dimensionality  
window_context = 50          # Context window size                                                                                    
min_word_count = 2   # Minimum word count                        
sample = 1e-3   # Downsample setting for frequent words
epoch = 10# Number of epochs 

In [18]:
from gensim.models.fasttext import FastText

# sg decides whether to use the skip-gram model (1) or CBOW (0)
ft_model_sg = FastText(sentences=x, window=window_context, vector_size=vec_size, 
                    min_count=min_word_count,sample=sample, sg=1, epochs=epoch)
                                 



# view similar words based on gensim's FastText model
similar_words_sg = {search_term: [item[0] for item in ft_model_sg.wv.most_similar([search_term], topn=5)]
                  for search_term in ['نحبك','افضل']}
print(similar_words_sg) 
#epoch = 2 # Number of epochs 


C:\Users\etudiant\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


{'نحبك': ['بنحبك', 'نحبوك', 'نحبو', 'نحبكم', 'يحبك'], 'افضل': ['هافضل', 'وافضل', 'افضضل', 'لافضل', 'فسافضل']}


# Savethemodels

In [20]:
ft_model_sg.save("./EmbeddingModels/ft_model_sg.model")


In [21]:
ft_model_cbow = FastText(sentences=x, window=window_context, vector_size=vec_size, 
                    min_count=min_word_count,sample=sample, sg=0, epochs=epoch)
similar_words_cbow = {search_term: [item[0] for item in ft_model_cbow.wv.most_similar([search_term], topn=5)]
                  for search_term in ['نحبك','افضل']}
print(similar_words_cbow) 

#epoch = 10# Number of epochs 


{'نحبك': ['جسدك', 'تهبك', 'غضبك', 'بقلبك', 'قلبك'], 'افضل': ['افضضل', 'كافضل', 'فافضل', 'فسافضل', 'هفضل']}


In [22]:
ft_model_cbow.save("./EmbeddingModels/ft_model_cbow.model")

In [23]:
print(ft_model_sg.wv.similarity(w1='نحبو', w2='نحبك'))
print(ft_model_sg.wv.similarity(w1='فضل', w2='فسافضل'))
print(ft_model_sg.wv.similarity(w1='افضل', w2='نموتو'))
print(ft_model_sg.wv.similarity(w1='سكر', w2='افضل'))

0.71330374
0.32955176
0.04373499
0.13556601


In [24]:
print(ft_model_cbow.wv.similarity(w1='نحبو', w2='نحبك'))
print(ft_model_cbow.wv.similarity(w1='ضل', w2='افضل'))
print(ft_model_cbow.wv.similarity(w1='نعشق', w2='نحبك'))
print(ft_model_cbow.wv.similarity(w1='نحبك', w2='غضبك'))

0.76219755
0.91874635
0.8537128
0.87432224


# Word2vec : 


In [25]:
from gensim.models import Word2Vec


In [26]:
# sg decides whether to use the skip-gram model (1) or CBOW (0)
w2v_model_sg = Word2Vec(sentences=x, window=window_context, vector_size=vec_size, 
                    min_count=min_word_count,sample=sample, sg=1, epochs=epoch)


w2v_model_cbow = Word2Vec(sentences=x, window=window_context, vector_size=vec_size, 
                    min_count=min_word_count,sample=sample, sg=0, epochs=epoch)


# view similar words based on gensim's Word2vec model
similar_words_sg = {search_term: [item[0] for item in w2v_model_sg.wv.most_similar([search_term], topn=5)]
                  for search_term in  ['نحبك','افضل']}
print(similar_words_sg) 

similar_words_cbow = {search_term: [item[0] for item in w2v_model_cbow.wv.most_similar([search_term], topn=5)]
                  for search_term in['نحبك','افضل']}
print(similar_words_cbow) 
#epoch = 10 # Number of epochs 



{'نحبك': ['نحبوك', 'يحفضك', 'برافوو', 'بركا', 'طلتك'], 'افضل': ['بالمانيا', 'بكثير', 'فالخبر', 'وذوقك', 'شابهه']}
{'نحبك': ['وهره', 'مناظله', 'السهريه', 'بتوفيق', 'معذبهم'], 'افضل': ['الافضل', 'بكثير', 'وافضل', 'بافضل', 'كازا']}


In [27]:
w2v_model_sg.save("./EmbeddingModels/w2v_model_sg.model")
w2v_model_cbow.save("./EmbeddingModels/w2v_model_cbow.model")

In [28]:
w2v_model_sg.wv.most_similar(positive="نحبك")

[('نحبوك', 0.7452232241630554),
 ('يحفضك', 0.722109854221344),
 ('برافوو', 0.7148568630218506),
 ('بركا', 0.7112698554992676),
 ('طلتك', 0.710512101650238),
 ('بكلهم', 0.7100936770439148),
 ('نشالله', 0.7097424864768982),
 ('السهريه', 0.707128643989563),
 ('بردتلي', 0.7070162892341614),
 ('ومحله', 0.7067701816558838)]

In [29]:
w2v_model_cbow.wv.most_similar(positive="افضل")

[('الافضل', 0.7576339244842529),
 ('بكثير', 0.6234630942344666),
 ('وافضل', 0.5824649930000305),
 ('بافضل', 0.5716922283172607),
 ('كازا', 0.5456104874610901),
 ('المعتاد', 0.5301099419593811),
 ('البندقيه', 0.5296929478645325),
 ('هاوس', 0.5228652954101562),
 ('باليرميتانو', 0.5206665396690369),
 ('شاروخان', 0.5204638242721558)]

In [30]:
def word_vector(tokens, size, model):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model.wv[word].reshape((1, size))
            count += 1
        except KeyError: # handling the case where the token is not in vocabulary
            
            continue
    if count != 0:
        vec /= count
    return vec

In [31]:
def get_matrix(data, vec_size, model):
    arrays = np.zeros((len(data), vec_size))

    for i in range(len(data)):
        arrays[i,:] = word_vector(data[i], vec_size , model)

    result = pd.DataFrame(arrays)
    return result

In [32]:
x_train_ft_sg = get_matrix(x_train,vec_size,ft_model_sg)
x_test_ft_sg = get_matrix(x_test,vec_size,ft_model_sg)

x_train_ft_cbow = get_matrix(x_train,vec_size,ft_model_cbow)
x_test_ft_cbow = get_matrix(x_test,vec_size,ft_model_cbow)

x_train_w2v_sg = get_matrix(x_train,vec_size,w2v_model_sg)
x_test_w2v_sg = get_matrix(x_test,vec_size,w2v_model_sg)

x_train_w2v_cbow = get_matrix(x_train,vec_size,w2v_model_cbow)
x_test_w2v_cbow = get_matrix(x_test,vec_size,w2v_model_cbow)

In [33]:
print(f"X_train  using FastText_cbow : {x_train_ft_cbow.shape}")
print(f"X_test   using FastText_cbow : {x_test_ft_cbow.shape}")

print(f"X_train  using FastText_cbow : {x_train_ft_cbow.shape}")
print(f"X_test   using FastText_cbow : {x_test_ft_cbow.shape}")

print(f"X_train using FastText_cbow : {x_train_ft_cbow.shape}")
print(f"X_test  using FastText_cbow : {x_test_ft_cbow.shape}")

print(f"X_train using FastText_cbow : {x_train_ft_cbow.shape}")
print(f"X_test  using FastText_cbow : {x_test_ft_cbow.shape}")


X_train  using FastText_cbow : (37398, 300)
X_test   using FastText_cbow : (12466, 300)
X_train  using FastText_cbow : (37398, 300)
X_test   using FastText_cbow : (12466, 300)
X_train using FastText_cbow : (37398, 300)
X_test  using FastText_cbow : (12466, 300)
X_train using FastText_cbow : (37398, 300)
X_test  using FastText_cbow : (12466, 300)


# Training the model : 


## SVM :

In [34]:
def svm(x_train, x_test, y_train, y_test):
    
    from sklearn import svm
    from sklearn.metrics import accuracy_score, classification_report

    # classify using support vector classifier
    svm = svm.SVC(kernel = 'linear', probability=True)

    # fit the SVC model based on the given training data
    prob = svm.fit(x_train, y_train).predict_proba(x_test)

    # perform classification and prediction on samples in x_test
    y_pred_svm = svm.predict(x_test)
    
    accuracy = accuracy_score(y_test, y_pred_svm) * 100 
    report = classification_report(y_test, y_pred_svm)
    
    return accuracy, report

In [37]:
accuracy_ft_sg,   report_ft_sg     = svm(x_train_ft_sg,x_test_ft_sg,y_train,y_test)
accuracy_ft_cbow, report_ft_cbow   = svm(x_train_ft_cbow,x_test_ft_cbow,y_train,y_test)

accuracy_w2v_sg,   report_w2v_sg   = svm(x_train_w2v_sg,x_test_w2v_sg,y_train,y_test)
accuracy_w2v_cbow, report_w2v_cbow = svm(x_train_w2v_cbow,x_test_w2v_cbow,y_train,y_test)

In [38]:
print("----------------------------------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------------------------------")

print(f"FastText_sg accuracy : {accuracy_ft_sg} %")
print(report_ft_sg)
print("----------------------------------------------------------------------------------------------------------------------")
print(f"FastText_cbow accuracy : {accuracy_ft_cbow} %")
print(report_ft_cbow)

print("----------------------------------------------------------------------------------------------------------------------")

print(f"Word2vec_sg accuracy : {accuracy_w2v_sg} %")
print(report_w2v_sg)
print("----------------------------------------------------------------------------------------------------------------------")
print(f"Word2vec_cbow accuracy : {accuracy_w2v_cbow} %")
print(report_w2v_cbow)


print("----------------------------------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------------------------------")


----------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------
FastText_sg accuracy : 84.1649286058078 %
              precision    recall  f1-score   support

           0       0.83      0.87      0.85      6233
           1       0.86      0.82      0.84      6233

    accuracy                           0.84     12466
   macro avg       0.84      0.84      0.84     12466
weighted avg       0.84      0.84      0.84     12466

----------------------------------------------------------------------------------------------------------------------
FastText_cbow accuracy : 79.43205519011713 %
              precision    recall  f1-score   support

           0       0.76      0.86      0.81      6233
 